In [1]:
import subprocess
import os
import torch
import sys, pathlib
from transformers import AutoModelForCausalLM, AutoTokenizer
from llm_blender.pair_ranker.pairrm import DebertaV2PairRM

from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset
import torch.nn.functional as F
from tqdm import tqdm
import re
import yaml

LOCAL_TRL_PARENT = "/root/autodl-tmp/Self_play_DRPO"
if LOCAL_TRL_PARENT not in sys.path:
    sys.path.insert(0, LOCAL_TRL_PARENT)

    
# now the import will use your local copy:
from trl import (
    DPOTrainer,
    DPOConfig,
    ModelConfig,
    DRPOTrainer,
    DRPOConfig,
)

from trl.trainer.drpo_utils import GPMwithRewardNetwork, estDPOStylePipeline, BTRewardNetwork, PariRMPipeline

# Load environment variables from /etc/network_turbo
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

def strip_prompt(prompt: str, text: str) -> str:
    """
    If `text` literally starts with `prompt` (ignoring leading/trailing
    whitespace), cut that prefix off and return the remainder.
    """
    p = prompt.strip()
    # Escaping safeguards punctuation / regex metacharacters
    pattern = r"^\s*" + re.escape(p) + r"\s*"
    return re.sub(pattern, "", text, count=1).lstrip()

/root/autodl-tmp/Self_play_DRPO/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [61]:
"""
ds_ultrafeed_train = load_dataset("openbmb/UltraFeedback", split = 'train', cache_dir="/root/autodl-tmp/dataset")
ds_ultrafeed_train = ds_ultrafeed_train.filter(lambda x: len(x['completions']) > 0)
def get_preferred(sample):
    prompt = sample['instruction']
    sample_completions = sample['completions']
    scores = [sample_completions[i]['overall_score'] for i in range(len(sample_completions))]
    responses = [sample_completions[i]['response'] for i in range(len(sample_completions))]
    preferred_index = scores.index(max(scores))
    dispreferred_index = scores.index(min(scores))
    preferred_ans = responses[preferred_index]
    dispreferred_ans = responses[dispreferred_index]
    return {
        'prompt':prompt,
        'preferred': preferred_ans,
        'dispreferred': dispreferred_ans,
    }
    
processed_dataset = ds_ultrafeed_train.map(get_preferred, remove_columns=ds_ultrafeed_train.column_names)
processed_dataset.push_to_hub("august66/DRPO_data_from_ultrafeed", split="train")
"""

'\nds_ultrafeed_train = load_dataset("openbmb/UltraFeedback", split = \'train\', cache_dir="/root/autodl-tmp/dataset")\nds_ultrafeed_train = ds_ultrafeed_train.filter(lambda x: len(x[\'completions\']) > 0)\ndef get_preferred(sample):\n    prompt = sample[\'instruction\']\n    sample_completions = sample[\'completions\']\n    scores = [sample_completions[i][\'overall_score\'] for i in range(len(sample_completions))]\n    responses = [sample_completions[i][\'response\'] for i in range(len(sample_completions))]\n    preferred_index = scores.index(max(scores))\n    dispreferred_index = scores.index(min(scores))\n    preferred_ans = responses[preferred_index]\n    dispreferred_ans = responses[dispreferred_index]\n    return {\n        \'prompt\':prompt,\n        \'preferred\': preferred_ans,\n        \'dispreferred\': dispreferred_ans,\n    }\n\nprocessed_dataset = ds_ultrafeed_train.map(get_preferred, remove_columns=ds_ultrafeed_train.column_names)\nprocessed_dataset.push_to_hub("august66/

In [ ]:
device = 'cuda'
model_name = 'Qwen/Qwen2.5-0.5B-Instruct'   # use 0.5B model to test for now 
cache_path = "/root/autodl-tmp/model_cache"
model_args = ModelConfig(model_name)
model_torch_dtype = torch.float16
model_kwargs = dict(
    revision = model_args.model_revision,
    torch_dtype = model_torch_dtype, 
    trust_remote_code = model_args.trust_remote_code,
)
lm_model_instance = AutoModelForCausalLM.from_pretrained(
    model_args.model_name_or_path,
    **model_kwargs,
    cache_dir = cache_path
).eval()

lm_model_tokenizer = AutoTokenizer.from_pretrained(
    model_args.model_name_or_path, 
    padding_side = 'left', 
    use_fast = True,
    trust_remote_code = model_args.trust_remote_code,
    cache_dir = cache_path
)

if not lm_model_tokenizer.pad_token:
    lm_model_tokenizer.pad_token = lm_model_tokenizer.eos_token

print (model_torch_dtype)

seed = 42
FIRST = 20_000
SECOND = 20_000
ds_prompt_reshuffle = ds_prompt_train.shuffle(seed=seed)
ds_prompt_split_1 = ds_prompt_reshuffle.select(range(FIRST))
ds_prompt_split_2 = ds_prompt_reshuffle.select(range(FIRST, FIRST + SECOND))
ds_prompt_split_3 = ds_prompt_reshuffle.select(range(FIRST + SECOND, len(ds_prompt_reshuffle)))


def collate_fn(batch):
    batch_prompts = [row["instruction"] for row in batch]
    enc = lm_model_tokenizer(
        batch_prompts,
        padding = True,
        truncation = True,
        max_length = 512,
        return_tensors = 'pt',
    )
    return enc.input_ids, enc.attention_mask, batch_prompts

loader = DataLoader(
    ds_prompt_split_1,
    batch_size = 256,
    shuffle = False,
    collate_fn = collate_fn,
    num_workers = 4
)

lm_model_instance.to(device)
prompt_completion_list = []
for input_ids, attentio_mask, prompts in tqdm(loader):
    input_ids, attentio_mask = input_ids.to(device), attentio_mask.to(device)
    outputs = lm_model_instance.generate(
        input_ids = input_ids,
        attention_mask = attentio_mask,
        max_new_tokens = 128,
        do_sample = True,
        temperature = 1.0,
        top_p = 1.0,
        num_return_sequences = 2,
        pad_token_id = lm_model_tokenizer.pad_token_id,
        eos_token_id = lm_model_tokenizer.eos_token_id,
    )
    decoded_output = lm_model_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    for i, prompt in enumerate(prompts):
        raw1, raw2 = decoded_output[2 * i : 2 * i + 2]

        # strip echoes
        resp1 = strip_prompt(prompt, raw1)
        resp2 = strip_prompt(prompt, raw2)

        prompt_completion_list.append({
            "instruction": prompt,
            "response": [resp1, resp2],
        })
ds_prompt_completion = Dataset.from_list(prompt_completion_list)


torch.float16


In [ ]:
seed = 42
FIRST = 20_000
SECOND = 20_000
ds_prompt_reshuffle = ds_prompt_train.shuffle(seed=seed)
ds_prompt_split_1 = ds_prompt_reshuffle.select(range(FIRST))
ds_prompt_split_2 = ds_prompt_reshuffle.select(range(FIRST, FIRST + SECOND))
ds_prompt_split_3 = ds_prompt_reshuffle.select(range(FIRST + SECOND, len(ds_prompt_reshuffle)))


def collate_fn(batch):
    batch_prompts = [row["instruction"] for row in batch]
    enc = lm_model_tokenizer(
        batch_prompts,
        padding = True,
        truncation = True,
        max_length = 512,
        return_tensors = 'pt',
    )
    return enc.input_ids, enc.attention_mask, batch_prompts

loader = DataLoader(
    ds_prompt_split_1,
    batch_size = 256,
    shuffle = False,
    collate_fn = collate_fn,
    num_workers = 4
)

lm_model_instance.to(device)
prompt_completion_list = []
for input_ids, attentio_mask, prompts in tqdm(loader):
    input_ids, attentio_mask = input_ids.to(device), attentio_mask.to(device)
    outputs = lm_model_instance.generate(
        input_ids = input_ids,
        attention_mask = attentio_mask,
        max_new_tokens = 128,
        do_sample = True,
        temperature = 1.0,
        top_p = 1.0,
        num_return_sequences = 2,
        pad_token_id = lm_model_tokenizer.pad_token_id,
        eos_token_id = lm_model_tokenizer.eos_token_id,
    )
    decoded_output = lm_model_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    for i, prompt in enumerate(prompts):
        raw1, raw2 = decoded_output[2 * i : 2 * i + 2]

        # strip echoes
        resp1 = strip_prompt(prompt, raw1)
        resp2 = strip_prompt(prompt, raw2)

        prompt_completion_list.append({
            "instruction": prompt,
            "response": [resp1, resp2],
        })
ds_prompt_completion = Dataset.from_list(prompt_completion_list)


In [ ]:
seed = 42
FIRST = 100
SECOND = 20_000
data_cache_path = "/root/autodl-tmp/dataset"
drpo_train = load_dataset("august66/DRPO_data_from_ultrafeed", split="train", cache_dir=data_cache_path)
drpo_train = drpo_train.rename_columns({
    "preferred": "a1",
    "dispreferred":"a2",
})

drpo_train_reshuffle = drpo_train.shuffle(seed=seed)
drpo_train_split_1 = drpo_train_reshuffle.select(range(FIRST))
drpo_train_split_2 = drpo_train_reshuffle.select(range(FIRST, FIRST + SECOND))
drpo_train_split_3 = drpo_train_reshuffle.select(range(FIRST + SECOND, len(drpo_train_reshuffle)))

In [21]:
device = 'cuda'
model_name = "Qwen/Qwen2-0.5B-Instruct"   # use 0.5B model to test for now 
cache_path = "/root/autodl-tmp/model_cache"
model_args = ModelConfig(model_name)
model_torch_dtype = torch.float16
model_args.trust_remote_code = True
model_kwargs = dict(
    revision = model_args.model_revision,
    torch_dtype = model_torch_dtype, 
    trust_remote_code = model_args.trust_remote_code,
)
lm_model_instance = AutoModelForCausalLM.from_pretrained(
    model_args.model_name_or_path,
    **model_kwargs,
    cache_dir = cache_path,
)

ref_model = AutoModelForCausalLM.from_pretrained(
    model_args.model_name_or_path,
    **model_kwargs,
    cache_dir = cache_path,
)

lm_model_tokenizer = AutoTokenizer.from_pretrained(
    model_args.model_name_or_path, 
    padding_side = 'left', 
    use_fast = True,
    trust_remote_code = model_args.trust_remote_code,
    cache_dir = cache_path
)

if not lm_model_tokenizer.pad_token:
    lm_model_tokenizer.pad_token = lm_model_tokenizer.eos_token

In [22]:
def clean_cuda():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

clean_cuda()

with open("/root/autodl-tmp/Self_play_DRPO/DRPO_scripts/hh/train_configs/config_gpm.yaml", "r") as f:
    training_args_config = yaml.safe_load(f)


training_args = DRPOConfig(
    **training_args_config
)

training_args.preference_model_id = 'llm-blender/PairRM-hf'

preference_pipeline = PariRMPipeline(
    model_name_or_path = training_args.preference_model_id,
)

    
trainer = DRPOTrainer(
    model=lm_model_instance,
    ref_model=ref_model,
    preference_model=preference_pipeline,
    train_dataset = drpo_train_split_1,
    processing_class=lm_model_tokenizer,
    args=training_args,
)
trainer.train()


Applying chat template to train dataset: 100%|██████████| 20000/20000 [00:01<00:00, 13528.83 examples/s]


after chat template dataset sample: {'prompt': 'Q: Please answer correctly the following question related to the paragraph below.   Was the water table increased or decreased in northeast Michigan in the twentieth century?  In the sixteenth century the State of Michigan had forests all over. But that has long been gone.  Especially, in the state\'s northeast part the vegetation has been cleared in the twentieth century.  Wooded areas were replaced by farm lands. All these farmlands were under heavy Irrigation. At one time, in the sixteenth century, plant species like elm and oak used to be plentiful. We still had some elms and oaks in the twentieth century, but they were not great many in number.  Hint: Groundwater salinization compromises buffering properties. Vegetation clearance, along with irrigation, causes serious issues. Irrigation increases the water table and mobilizes salts, and vegetation clearance allows it to come in contact with water habitats and vegetation. This stresse

Tokenizing train dataset: 100%|██████████| 20000/20000 [00:43<00:00, 456.58 examples/s]


RuntimeError: WandbCallback requires wandb to be installed. Run `pip install wandb`.

In [17]:
training_args

DRPOConfig(output_dir='./output/hh/gpm/', overwrite_output_dir=False, do_train=False, do_eval=False, do_predict=False, eval_strategy=<IntervalStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=16, eval_accumulation_steps=None, eval_delay=0, torch_empty_cache_steps=1, learning_rate=5e-07, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=0.25, num_train_epochs=1, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, lr_scheduler_kwargs={}, warmup_ratio=0.0, warmup_steps=0, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='./output/hh/gpm/runs/Jun21_13-51-13_autodl-container-7ed14e9210-e5b26c77', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=5, logging_nan_inf_filter=True, save_strategy=<SaveStrategy.STEPS: 'steps'>

In [16]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import llm_blender
blender = llm_blender.Blender()
# Load Ranker
blender.loadranker("llm-blender/PairRM") # load ranker checkpoint
inputs = ["hello!"]
candidates_A = ["hi!"]
candidates_B = ["f**k off!"]
logits = blender.compare(inputs, candidates_A, candidates_B, return_logits=True, mode="[A,B]")
comparison_results = logits > 0
print(logits)
# [ 1.9   -1.255]
print(comparison_results)
# tensor([ True, False], device='cuda:0'), which means whether candidate A is better than candidate B for each input


Successfully loaded ranker from  /root/.cache/huggingface/hub/llm-blender/PairRM


Ranking candidates: 100%|██████████| 1/1 [00:00<00:00, 29.44it/s]

[1.898]
[ True]


In [2]:
pairrm = PariRMPipeline(
    model_name_or_path = 'llm-blender/PairRM-hf',
)

In [3]:
inputs = ["hello!", "I love you!"]
candidates_A = ["hi!", "I hate you!"]
candidates_B = ["f**k off!", "I love you, too!"]

In [4]:
torch.tensor(pairrm(inputs, candidates_A, candidates_B)).sigmoid()

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([0.8699, 0.2219])

In [8]:
type(pairrm.model)

llm_blender.pair_ranker.pairrm.DebertaV2PairRM

In [6]:
isinstance(pairrm.model, DebertaV2PairRM)

True

In [6]:
from trl.trainer.drpo_utils import get_preference_score
get_preference_score(pairrm, candidates_A, candidates_B, inputs = inputs, kwargs = {})

tensor([0.8699, 0.2219])

In [ ]:
drpo_train_split_1_test = drpo_train_split_1.select(range(100))

In [12]:
get_preference_score(pairrm, drpo_train_split_1_test['preferred'], drpo_train_split_1_test['dispreferred'], inputs = drpo_train_split_1_test['prompt'], kwargs = {})

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.95 GiB. GPU 0 has a total capacity of 23.68 GiB of which 734.69 MiB is free. Process 126585 has 22.96 GiB memory in use. Of the allocated memory 22.29 GiB is allocated by PyTorch, and 384.39 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [19]:
drpo_train

Dataset({
    features: ['prompt', 'preferred', 'dispreferred'],
    num_rows: 63966
})